<a href="https://colab.research.google.com/github/Momo227/nlp_practice/blob/Chapter4/Chapter4_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**単語分割**：テキストを単語に分割する処理
* 自然言語をする際によく使われる→入力を単語レベルで扱うから
* 形態素解解析器を用いて行う
* 語彙数を減らすために単語を原型にすることもある

形態素解析する際の問題点

・デフォルトでは新語の解析に強くない（特にWebには新語が多数含まれているのでかなり深刻となる。）
    
   →形態素解析器に辞書を追加することである程度解決できる

辞書の例：NEologd


Pythonから使える主な形態素解析器

* MeCab
* Juman++
* Janome

今回は
* 形態素解析器 : Janome
* クラス：Tokenizer
* メソッド：tokenize

で行う。

In [22]:
pip install janome

In [29]:
from janome.tokenizer import Tokenizer

In [30]:
text = '彼女と国立新美術館へ行った。'

In [34]:
t = Tokenizer()

In [35]:
for token in t.tokenize(text):
    print(token)

彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
と	助詞,格助詞,一般,*,*,*,と,ト,ト
国立	名詞,一般,*,*,*,*,国立,コクリツ,コクリツ
新	接頭詞,名詞接続,*,*,*,*,新,シン,シン
美術館	名詞,一般,*,*,*,*,美術館,ビジュツカン,ビジュツカン
へ	助詞,格助詞,一般,*,*,*,へ,ヘ,エ
行っ	動詞,自立,*,*,五段・カ行促音便,連用タ接続,行く,イッ,イッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。


**解析結果**

左から
* 表層形
* 品詞
* 品詞細分類1
* 品詞細分類2
* 品詞細分類3
* 活用形
* 活用型
* 原形
* 読み
* 発音

Toenizerにwakati=Trueと設定すれば、単語のリストのみを得ることができる。

In [38]:
t = Tokenizer(wakati=True)

In [51]:
t.tokenize(text)

<generator object Tokenizer.__tokenize_stream at 0x7f3aada7acd0>

In [52]:
pip list | grep Janome

Janome                        0.4.1         


janome v0.4.0の変更点

・Tokenizer はストリームモードのみサポートするように変更
本バージョンより，Tokenizer は，メモリリークが起こりにくいストリームモードのみサポートします。具体的には，tokenize() メソッドの戻り値はリストではなくイテレータ（generator）のみとなり，また stream オプションは廃止となります。


v0.3.x　なら

['彼女', 'と', '国立', '新', '美術館', 'へ', '行っ', 'た', '。']

と表示されるはず。

形態素解析結果かた特定の品詞だけを抽出する場合。

* Analyzer：形態素解析の前処理や後処理を行えるクラス。
* POSKeepFilter：指定した品詞を抽出するためのクラス。

を使う。

手順


1.   Analyzerにフィルタを渡して初期化。
2.   analyzeメソッドを使ってテキストを解析。



In [41]:
from janome.analyzer import Analyzer

In [42]:
from janome.tokenfilter import POSKeepFilter

In [ ]:
#ファイルの定義

In [43]:
token_filters = [POSKeepFilter('名詞')]

In [44]:
a = Analyzer(token_filters=token_filters)

In [46]:
for token in a.analyze(text):
    print(token)

彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
国立	名詞,一般,*,*,*,*,国立,コクリツ,コクリツ
美術館	名詞,一般,*,*,*,*,美術館,ビジュツカン,ビジュツカン


解析結果は名詞だけ抽出された。

Analyzerには複数のフィルタを渡すことが可能。

**辞書に単語を追加する方法**

その前に

辞書の定義内容
* 表層形
* 左文脈ID
* 右文脈ID
* コスト：その単語がどれだけ出現しやすいか
* 品詞
* 品詞細分類1
* 品詞細分類2
* 品詞細分類3
* 活用形
* 活用型
* 原形
* 読み
* 発音


In [47]:
from janome.tokenizer import Tokenizer

In [48]:
t = Tokenizer(udic='userdic.csv', udic_enc='utf8')

In [49]:
text = '彼女と国立新美術館へ行った。'

In [50]:
for token in t.tokenize(text):
  print(token)

彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
と	助詞,並立助詞,*,*,*,*,と,ト,ト
国立新美術館	名詞,固有名詞,一般,*,*,*,国立新美術館,コクリツシンビジュツカン,コクリツシンビジュツカン
へ	助詞,格助詞,一般,*,*,*,へ,ヘ,エ
行っ	動詞,自立,*,*,五段・カ行促音便,連用タ接続,行く,イッ,イッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。


「国立新美術館」が１単語として認識されるようになった。